In [1]:
import numpy as np
import pandas as pd
import json


In [2]:
'''
Example python notebook to show how testing a model with generated profiles and requests would look like
The user (student) will be referenced by their unique id
The ID will be used to retrieve the student's profile information which will be loaded into a dataframe
The student's request will also be retrieved
The model will use the request information as well as the dataframe of profiles to determine the best profiles marked as 
"tutor" or "Both" to tutor the student. The top 5 or 10 user ids (from the profiles) will be output and we'll check
the groundtruth ids to see how many optimal tutors were suggested correctly

'''

'\nExample python notebook to show how testing a model with generated profiles and requests would look like\nThe user (student) will be referenced by their unique id\nThe ID will be used to retrieve the student\'s profile information which will be loaded into a dataframe\nThe student\'s request will also be retrieved\nThe model will use the request information as well as the dataframe of profiles to determine the best profiles marked as \n"tutor" or "Both" to tutor the student. The top 5 or 10 user ids (from the profiles) will be output and we\'ll check\nthe groundtruth ids to see how many optimal tutors were suggested correctly\n\n'

In [3]:
#id = "some userid for the profile"
id = "557610956080"

# read in profiles data
profiles = pd.read_json("profiles.json")

In [4]:
profiles.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
Major,MGMT,Engineering,Engineering,Engineering,Engineering,Science,Engineering,Science,Science,Science,...,Engineering,Info Technology,Science,Engineering,Science,Engineering,Engineering,MGMT,Science,Engineering
Role,Student,Tutor,Both,Both,Both,Both,Both,Both,Both,Both,...,Both,Both,Tutor,Both,Both,Both,Student,Both,Both,Both
Degree,Graduate,Graduate,Undergraduate,Undergraduate,Graduate,Graduate,Undergraduate,Undergraduate,Undergraduate,Undergraduate,...,Undergraduate,Undergraduate,Undergraduate,Undergraduate,Graduate,Undergraduate,Undergraduate,Undergraduate,Undergraduate,Undergraduate
Bio,Generic Bio,Generic Bio,Generic Bio,Generic Bio,Generic Bio,Generic Bio,Generic Bio,Generic Bio,Generic Bio,Generic Bio,...,Generic Bio,Generic Bio,Generic Bio,Generic Bio,Generic Bio,Generic Bio,Generic Bio,Generic Bio,Generic Bio,Generic Bio


In [5]:
profiles = profiles.transpose()
profiles.head()

,Major,Role,Degree,Bio
0,MGMT,Student,Graduate,Generic Bio
1,Engineering,Tutor,Graduate,Generic Bio
2,Engineering,Both,Undergraduate,Generic Bio
3,Engineering,Both,Undergraduate,Generic Bio
4,Engineering,Both,Graduate,Generic Bio


In [7]:
#read in requests data
requests = pd.read_json("requests.json")
requests.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
status,close,close,open,open,close,close,close,open,open,open,...,open,open,close,close,open,close,close,close,open,open
state,CANCELLED ASSIGNED,FULFILLED,FULFILLED,CANCELLED ASSIGNED,CANCELLED ASSIGNED,CANCELLED ASSIGNED,FULFILLED,CANCELLED ASSIGNED,FULFILLED,ASSIGNED,...,CANCELLED ASSIGNED,ASSIGNED,CANCELLED ASSIGNED,FULFILLED,ASSIGNED,FULFILLED,ASSIGNED,ASSIGNED,CANCELLED ASSIGNED,FULFILLED
student,190410739211,501233125204,432076874077,473135247866,638764653001,141061564530,335136714316,481716264942,853495805899,663058207023,...,906844210631,807244289301,887867136559,738456229616,205570753657,426687251127,620197194595,894106900138,611143700687,192281564003
tutor,514507470926,596345891396,577226305568,555940543602,117663491614,481420883160,721897407744,814160786684,113731104729,948986014109,...,150492658601,121081402229,276542601017,633523973230,580341882048,550783517791,759532029453,420226765933,196131462672,227277317228
pinged_tutors,262178723406,557610956080,536000238102,690965507051,848515983729,558850183004,304111055536,409375406822,639793927476,526246366079,...,167233882116,450709992273,953533055204,391124466482,823550222932,424788232626,572407993694,656536465489,922894026397,655949905375


In [8]:
requests = requests.transpose()
requests.head()

,status,state,student,tutor,pinged_tutors,accepted_tutors,denied_tutors,cancelled_tutors,stu_rating,tut_rating
0,close,CANCELLED ASSIGNED,190410739211,514507470926,262178723406,514507470926,487704839823,843412692102,1,1
1,close,FULFILLED,501233125204,596345891396,557610956080,596345891396,375755529049,141365810034,5,2
2,open,FULFILLED,432076874077,577226305568,536000238102,577226305568,191913380917,967898307536,1,5
3,open,CANCELLED ASSIGNED,473135247866,555940543602,690965507051,555940543602,424788232626,518332650411,2,3
4,close,CANCELLED ASSIGNED,638764653001,117663491614,848515983729,117663491614,959066110193,773756748376,4,5


In [ ]:
# get the request row that has the userid as a student feature
